# IA for HumanForYou

## Context

L'entreprise de produits pharmaceutiques HumanForYou basée en Inde emploie environ 4000 personnes. Cependant, chaque année elle subit un turn-over d'environ 15% de ses employés nécessitant de retrouver des profils similaires sur le marché de l'emploi.

La direction trouve que ce niveau de turn-over n'est pas bon pour l'entreprise car :

Les projets sur lesquels étaient les employés quittant la société prennent du retard ce qui nuit à la réputation de l'entreprise auprès de ses clients et partenaires.

Un service de ressources humaines de taille conséquente doit être conservé car il faut avoir les moyens de trouver les nouvelles recrues.

Du temps est perdu à l'arrivée des nouveaux employés car ils doivent très souvent être formés et ont besoin de temps pour devenir pleinement opérationnels dans leur nouvel environnement.

Le direction fait donc appel à vous, spécialistes de l'analyse de données, pour déterminer les facteurs ayant le plus d'influence sur ce taux de turn-over et lui proposer des modèles afin d'avoir des pistes d'amélioration pour donner à leurs employés l'envie de rester.

Données fournies
Un certain nombre de données concernant les employés vous a donc été transmis par le service des ressources humaines.

Il s'agit de fichiers textes au format CSV.

Les données ont été anonymisées : un employé de l'entreprise sera représenté par le même EmployeeID dans l'ensemble des fichiers qui suivent.

In [2]:
# imports
import numpy as np
import os
import pandas as pd

## Import des données

On va importer toutes les données dans des tableaux

In [18]:
GENERAL_PATH = os.path.join("Datasets")
IN_OUT_TIME_PATH = os.path.join("Datasets", "in_out_time")

MANAGER_SURVEY_CSV = "manager_survey_data.csv"
GENERAL_CSV = "general_data.csv"
IN_TIME_CSV = "in_time.csv"
OUT_TIME_CSV = "out_time.csv"

def load_data(general_path, csv):
    csv_path = os.path.join(general_path, csv)
    df = pd.read_csv(csv_path, sep=",", header = 0)
    return df

In [33]:
general = load_data(GENERAL_PATH, GENERAL_CSV)
in_time = load_data(IN_OUT_TIME_PATH, IN_TIME_CSV)
out_time = load_data(IN_OUT_TIME_PATH, OUT_TIME_CSV)
manager_survey = load_data(GENERAL_PATH, MANAGER_SURVEY_CSV)

## Check des données

On va regarder les prmières lignes des données

In [34]:
general.head()

,Age,Attrition,BusinessTravel,Department,DistanceFromHome,Education,EducationField,EmployeeCount,EmployeeID,Gender,...,NumCompaniesWorked,Over18,PercentSalaryHike,StandardHours,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,YearsAtCompany,YearsSinceLastPromotion,YearsWithCurrManager
0,51,No,Travel_Rarely,Sales,6,2,Life Sciences,1,1,Female,...,1.0,Y,11,8,0,1.0,6,1,0,0
1,31,Yes,Travel_Frequently,Research & Development,10,1,Life Sciences,1,2,Female,...,0.0,Y,23,8,1,6.0,3,5,1,4
2,32,No,Travel_Frequently,Research & Development,17,4,Other,1,3,Male,...,1.0,Y,15,8,3,5.0,2,5,0,3
3,38,No,Non-Travel,Research & Development,2,5,Life Sciences,1,4,Male,...,3.0,Y,11,8,3,13.0,5,8,7,5
4,32,No,Travel_Rarely,Research & Development,10,1,Medical,1,5,Male,...,4.0,Y,12,8,2,9.0,2,6,0,4


In [35]:
in_time.head()

,Unnamed: 0,2015-01-01,2015-01-02,2015-01-05,2015-01-06,2015-01-07,2015-01-08,2015-01-09,2015-01-12,2015-01-13,...,2015-12-18,2015-12-21,2015-12-22,2015-12-23,2015-12-24,2015-12-25,2015-12-28,2015-12-29,2015-12-30,2015-12-31
0,1,NaN,2015-01-02 09:43:45,2015-01-05 10:08:48,2015-01-06 09:54:26,2015-01-07 09:34:31,2015-01-08 09:51:09,2015-01-09 10:09:25,2015-01-12 09:42:53,2015-01-13 10:13:06,...,NaN,2015-12-21 09:55:29,2015-12-22 10:04:06,2015-12-23 10:14:27,2015-12-24 10:11:35,NaN,2015-12-28 10:13:41,2015-12-29 10:03:36,2015-12-30 09:54:12,2015-12-31 10:12:44
1,2,NaN,2015-01-02 10:15:44,2015-01-05 10:21:05,NaN,2015-01-07 09:45:17,2015-01-08 10:09:04,2015-01-09 09:43:26,2015-01-12 10:00:07,2015-01-13 10:43:29,...,2015-12-18 10:37:17,2015-12-21 09:49:02,2015-12-22 10:33:51,2015-12-23 10:12:10,NaN,NaN,2015-12-28 09:31:45,2015-12-29 09:55:49,2015-12-30 10:32:25,2015-12-31 09:27:20
2,3,NaN,2015-01-02 10:17:41,2015-01-05 09:50:50,2015-01-06 10:14:13,2015-01-07 09:47:27,2015-01-08 10:03:40,2015-01-09 10:05:49,2015-01-12 10:03:47,2015-01-13 10:21:26,...,2015-12-18 10:15:14,2015-12-21 10:10:28,2015-12-22 09:44:44,2015-12-23 10:15:54,2015-12-24 10:07:26,NaN,2015-12-28 09:42:05,2015-12-29 09:43:36,2015-12-30 09:34:05,2015-12-31 10:28:39
3,4,NaN,2015-01-02 10:05:06,2015-01-05 09:56:32,2015-01-06 10:11:07,2015-01-07 09:37:30,2015-01-08 10:02:08,2015-01-09 10:08:12,2015-01-12 10:13:42,2015-01-13 09:53:22,...,2015-12-18 10:17:38,2015-12-21 09:58:21,2015-12-22 10:04:25,2015-12-23 10:11:46,2015-12-24 09:43:15,NaN,2015-12-28 09:52:44,2015-12-29 09:33:16,2015-12-30 10:18:12,2015-12-31 10:01:15
4,5,NaN,2015-01-02 10:28:17,2015-01-05 09:49:58,2015-01-06 09:45:28,2015-01-07 09:49:37,2015-01-08 10:19:44,2015-01-09 10:00:50,2015-01-12 10:29:27,2015-01-13 09:59:32,...,2015-12-18 09:58:35,2015-12-21 10:03:41,2015-12-22 10:10:30,2015-12-23 10:13:36,2015-12-24 09:44:24,NaN,2015-12-28 10:05:15,2015-12-29 10:30:53,2015-12-30 09:18:21,2015-12-31 09:41:09


In [36]:
out_time.head()

,Unnamed: 0,2015-01-01,2015-01-02,2015-01-05,2015-01-06,2015-01-07,2015-01-08,2015-01-09,2015-01-12,2015-01-13,...,2015-12-18,2015-12-21,2015-12-22,2015-12-23,2015-12-24,2015-12-25,2015-12-28,2015-12-29,2015-12-30,2015-12-31
0,1,NaN,2015-01-02 16:56:15,2015-01-05 17:20:11,2015-01-06 17:19:05,2015-01-07 16:34:55,2015-01-08 17:08:32,2015-01-09 17:38:29,2015-01-12 16:58:39,2015-01-13 18:02:58,...,NaN,2015-12-21 17:15:50,2015-12-22 17:27:51,2015-12-23 16:44:44,2015-12-24 17:47:22,NaN,2015-12-28 18:00:07,2015-12-29 17:22:30,2015-12-30 17:40:56,2015-12-31 17:17:33
1,2,NaN,2015-01-02 18:22:17,2015-01-05 17:48:22,NaN,2015-01-07 17:09:06,2015-01-08 17:34:04,2015-01-09 16:52:29,2015-01-12 17:36:48,2015-01-13 18:00:13,...,2015-12-18 18:31:28,2015-12-21 17:34:16,2015-12-22 18:16:35,2015-12-23 17:38:18,NaN,NaN,2015-12-28 17:08:38,2015-12-29 17:54:46,2015-12-30 18:31:35,2015-12-31 17:40:58
2,3,NaN,2015-01-02 16:59:14,2015-01-05 17:06:46,2015-01-06 16:38:32,2015-01-07 16:33:21,2015-01-08 17:24:22,2015-01-09 16:57:30,2015-01-12 17:28:54,2015-01-13 17:21:25,...,2015-12-18 17:02:23,2015-12-21 17:20:17,2015-12-22 16:32:50,2015-12-23 16:59:43,2015-12-24 16:58:25,NaN,2015-12-28 16:43:31,2015-12-29 17:09:56,2015-12-30 17:06:25,2015-12-31 17:15:50
3,4,NaN,2015-01-02 17:25:24,2015-01-05 17:14:03,2015-01-06 17:07:42,2015-01-07 16:32:40,2015-01-08 16:53:11,2015-01-09 17:19:47,2015-01-12 17:13:37,2015-01-13 17:11:45,...,2015-12-18 17:55:23,2015-12-21 16:49:09,2015-12-22 17:24:00,2015-12-23 17:36:35,2015-12-24 16:48:21,NaN,2015-12-28 17:19:34,2015-12-29 16:58:16,2015-12-30 17:40:11,2015-12-31 17:09:14
4,5,NaN,2015-01-02 18:31:37,2015-01-05 17:49:15,2015-01-06 17:26:25,2015-01-07 17:37:59,2015-01-08 17:59:28,2015-01-09 17:44:08,2015-01-12 18:51:21,2015-01-13 18:14:58,...,2015-12-18 17:52:48,2015-12-21 17:43:35,2015-12-22 18:07:57,2015-12-23 18:00:49,2015-12-24 17:59:22,NaN,2015-12-28 17:44:59,2015-12-29 18:47:00,2015-12-30 17:15:33,2015-12-31 17:42:14


In [37]:
manager_survey.head()

,EmployeeID,JobInvolvement,PerformanceRating
0,1,3,3
1,2,2,4
2,3,3,3
3,4,2,3
4,5,3,3


## Merge des données en un seul tableau

In [38]:
merged_data = general.merge(manager_survey, how='left', on='EmployeeID')

In [45]:
time_frame = in_time.merge(out_time, how='left', on='Unnamed: 0')

In [41]:
merged_data.value_counts("JobRole")

JobRole
Sales Executive              978
Research Scientist           876
Laboratory Technician        777
Manufacturing Director       435
Healthcare Representative    393
Manager                      306
Sales Representative         249
Research Director            240
Human Resources              156
dtype: int64

In [48]:
def rename_head(df):
    for col in df.columns:
        print(col)

In [49]:
rename_head(in_time)

Unnamed: 0
2015-01-01
2015-01-02
2015-01-05
2015-01-06
2015-01-07
2015-01-08
2015-01-09
2015-01-12
2015-01-13
2015-01-14
2015-01-15
2015-01-16
2015-01-19
2015-01-20
2015-01-21
2015-01-22
2015-01-23
2015-01-26
2015-01-27
2015-01-28
2015-01-29
2015-01-30
2015-02-02
2015-02-03
2015-02-04
2015-02-05
2015-02-06
2015-02-09
2015-02-10
2015-02-11
2015-02-12
2015-02-13
2015-02-16
2015-02-17
2015-02-18
2015-02-19
2015-02-20
2015-02-23
2015-02-24
2015-02-25
2015-02-26
2015-02-27
2015-03-02
2015-03-03
2015-03-04
2015-03-05
2015-03-06
2015-03-09
2015-03-10
2015-03-11
2015-03-12
2015-03-13
2015-03-16
2015-03-17
2015-03-18
2015-03-19
2015-03-20
2015-03-23
2015-03-24
2015-03-25
2015-03-26
2015-03-27
2015-03-30
2015-03-31
2015-04-01
2015-04-02
2015-04-03
2015-04-06
2015-04-07
2015-04-08
2015-04-09
2015-04-10
2015-04-13
2015-04-14
2015-04-15
2015-04-16
2015-04-17
2015-04-20
2015-04-21
2015-04-22
2015-04-23
2015-04-24
2015-04-27
2015-04-28
2015-04-29
2015-04-30
2015-05-01
2015-05-04
2015-05-05
2015-05-06

In [85]:
header = in_time.columns.to_list()

In [100]:
header[0] = 'EmployeeId'

for i, col in enumerate(header[1:]):
    print(i, col)
    header[i+1] = col + "_IN"

0 2015-01-01IN_IN
1 2015-01-02IN_IN
2 2015-01-05IN_IN
3 2015-01-06IN_IN
4 2015-01-07IN_IN
5 2015-01-08IN_IN
6 2015-01-09IN_IN
7 2015-01-12IN_IN
8 2015-01-13IN_IN
9 2015-01-14IN_IN
10 2015-01-15IN_IN
11 2015-01-16IN_IN
12 2015-01-19IN_IN
13 2015-01-20IN_IN
14 2015-01-21IN_IN
15 2015-01-22IN_IN
16 2015-01-23IN_IN
17 2015-01-26IN_IN
18 2015-01-27IN_IN
19 2015-01-28IN_IN
20 2015-01-29IN_IN
21 2015-01-30IN_IN
22 2015-02-02IN_IN
23 2015-02-03IN_IN
24 2015-02-04IN_IN
25 2015-02-05IN_IN
26 2015-02-06IN_IN
27 2015-02-09IN_IN
28 2015-02-10IN_IN
29 2015-02-11IN_IN
30 2015-02-12IN_IN
31 2015-02-13IN_IN
32 2015-02-16IN_IN
33 2015-02-17IN_IN
34 2015-02-18IN_IN
35 2015-02-19IN_IN
36 2015-02-20IN_IN
37 2015-02-23IN_IN
38 2015-02-24IN_IN
39 2015-02-25IN_IN
40 2015-02-26IN_IN
41 2015-02-27IN_IN
42 2015-03-02IN_IN
43 2015-03-03IN_IN
44 2015-03-04IN_IN
45 2015-03-05IN_IN
46 2015-03-06IN_IN
47 2015-03-09IN_IN
48 2015-03-10IN_IN
49 2015-03-11IN_IN
50 2015-03-12IN_IN
51 2015-03-13IN_IN
52 2015-03-16IN_IN
53 

In [98]:
header

['EmployeeId',
 '2015-01-01IN_IN',
 '2015-01-02IN_IN',
 '2015-01-05IN_IN',
 '2015-01-06IN_IN',
 '2015-01-07IN_IN',
 '2015-01-08IN_IN',
 '2015-01-09IN_IN',
 '2015-01-12IN_IN',
 '2015-01-13IN_IN',
 '2015-01-14IN_IN',
 '2015-01-15IN_IN',
 '2015-01-16IN_IN',
 '2015-01-19IN_IN',
 '2015-01-20IN_IN',
 '2015-01-21IN_IN',
 '2015-01-22IN_IN',
 '2015-01-23IN_IN',
 '2015-01-26IN_IN',
 '2015-01-27IN_IN',
 '2015-01-28IN_IN',
 '2015-01-29IN_IN',
 '2015-01-30IN_IN',
 '2015-02-02IN_IN',
 '2015-02-03IN_IN',
 '2015-02-04IN_IN',
 '2015-02-05IN_IN',
 '2015-02-06IN_IN',
 '2015-02-09IN_IN',
 '2015-02-10IN_IN',
 '2015-02-11IN_IN',
 '2015-02-12IN_IN',
 '2015-02-13IN_IN',
 '2015-02-16IN_IN',
 '2015-02-17IN_IN',
 '2015-02-18IN_IN',
 '2015-02-19IN_IN',
 '2015-02-20IN_IN',
 '2015-02-23IN_IN',
 '2015-02-24IN_IN',
 '2015-02-25IN_IN',
 '2015-02-26IN_IN',
 '2015-02-27IN_IN',
 '2015-03-02IN_IN',
 '2015-03-03IN_IN',
 '2015-03-04IN_IN',
 '2015-03-05IN_IN',
 '2015-03-06IN_IN',
 '2015-03-09IN_IN',
 '2015-03-10IN_IN',
 '201